In [ ]:
# import sklearn.datasets
import pandas as pd
import itertools
import matplotlib.pyplot as plt

# Sort function
take = lambda n, iterable: list(itertools.islice(iterable, n)) 
ascending = lambda array: dict(reversed(sorted(array.items(), key=lambda item: item[1])))

class Analyzer:
    def __init__(self, sourcePath):
        self.data = pd.read_csv(sourcePath)
        self.indexes = self.data['index']
        self.labels = self.data['label']
        self.texts = self.data['text']

    def processData(self):
        
        # Create the slots for labels and grams
        self.words = {}
        # Label 0,1
        for i in range(3):
            # 1,2,3 - Grams
            self.words[i] = {}
            
            for j in range(3):
                self.words[i][j] = {}

        # print(self.texts)
        for i,text in enumerate(self.texts):
            # if i > 5:
            #     break
            # print(i,text)
            prev, prevPrev, current = "", "", ""
            for j, word in enumerate(text.split(" ")):
                prevPrev = prev
                prev = current
                current = word
                
                # 1-Gram
                if current not in self.words[self.labels[i]][0]:
                    self.words[self.labels[i] ][0][current] = 1
                else:
                    self.words[self.labels[i] ][0][current] = self.words[self.labels[i] ][0].get(current) + 1

                # Labels together
                if current not in self.words[2][0]:
                    self.words[2][0][current] = 1
                else:
                    self.words[2][0][current] = self.words[2][0].get(current) + 1

                # 2-Gram
                if j > 0:
                    act2 = prev + ' ' + current
                    # print('act2', act2)
                    if act2 not in self.words[self.labels[i]][1]:
                        self.words[self.labels[i] ][1][act2] = 1
                    else:
                        self.words[self.labels[i] ][1][act2] = self.words[self.labels[i] ][1].get(act2) + 1

                    # Labels together
                    if act2 not in self.words[2][1]:
                        self.words[2][1][act2] = 1
                    else:
                        self.words[2][1][act2] = self.words[2][1].get(act2) + 1

                # 3-Gram
                if j > 1:
                    act3 = prevPrev + ' ' + prev + ' ' + current
                    # print('act3', act3)
                    if act3 not in self.words[self.labels[i]][2]:
                        self.words[self.labels[i] ][2][act3] = 1
                    else:
                        self.words[self.labels[i] ][2][act3] = self.words[self.labels[i] ][2].get(act3) + 1
                    
                    # Labels together
                    if act3 not in self.words[2][2]:
                        self.words[2][2][act3] = 1
                    else:
                        self.words[2][2][act3] = self.words[2][2].get(act3) + 1
        print("process finished")
        # print(self.words[0][1])

    def sortData(self):
        self.sortedAscending = {}
        # Label 0,1
        for i in range(3):
            # 1,2,3 - Grams

            self.sortedAscending[i] = {}
            for j in range(3):
                # Sort all the words
                self.sortedAscending[i][j] = ascending( self.words[i][j] )
        print("data sorted")

    # Check the correctness by the first element's number
    # in each subset (labeled with 1 and 0) and the whole.
    def checkCounting(self):
        # Check the first X item
        checkedItems = 5
        passed = True

        for i in range(checkedItems):
            if not passed:
                break

            # Grams
            for j in range(3):
                # Get the most frequent word of all in J - Gram
                word = list(self.sortedAscending[2][j].items())[i][0]

                # Counters
                total = list(self.sortedAscending[2][j].items())[i][1]
                label1 = self.sortedAscending[0][j].get(word)
                label2 = self.sortedAscending[1][j].get(word)
                if label1 + label2 != total:
                    print('Checker failed: with the word ' + word)
                    print(label1 + label2, '!=', total, ' difference:', abs( total - label1 - label2))
                    print('The whole counter is not equal to the addition of label 0 and 1\'s counter.')
                    passed = False
                    break
                    
        if passed:
            print('Checking passed!')

    def lengthAnalysis(self):
        # Is there a collaration between the avarage length and label?
        self.lengths = { 0: {}, 1: {} }
        summ = [0,0]
        count = [0,0]
        for i, text in self.texts.items():
            # Simple avarage length
            la = self.labels[i]
            summ[la] += len(text)
            count[la] += 1
            # Number of texts by length
            if round(len(text),-1) in self.lengths[la]:
                self.lengths[la][round(len(text),-1)] += 1
            else:
                self.lengths[la][round(len(text),-1)] = 1

        plt.plot( self.lengths[1].keys(), self.lengths[1].values(), 'ro', markersize=2)
        plt.plot( self.lengths[0].keys(), self.lengths[0].values(), 'bo', markersize=2)
        plt.xlabel('length of reviews')
        plt.ylabel('number of reviews')
        print('Label 0 - Number of reviews:', count[0], '; The avarage length:', round(summ[0]/count[0], 2) )
        print('Label 1 - Number of reviews:', count[1], '; The avarage length:', round(summ[1]/count[1], 2) )
        print()
        print('Label 1 - red; Label 0 - blue')
        plt.show()

    def lengthCorrelation(self):
        lengths = [(lambda x: len(x[1]))(x) for x in self.texts.items()]
        lenLab = pd.DataFrame( list(zip(lengths, self.labels)), columns=["length","labels"])
        print(lenLab.corr())

    def printFrequent(self,n):
        # The first 5 most frequent elements
        for j in range(3):
            for i in range(3):
                if i == 2:
                    # Together the labels
                    print('Counting the whole G-' + str(j+1))
                else:
                    # i - Labels, j - Grams
                    print('L-' + str(i) + ' G-' + str(j+1) + ' (in ascending order): ')

                words = ''
                scores = ''
                for k,word in enumerate(take(n, train.sortedAscending[i][j].items())):
                    if k == 0:
                        words += '\'' + word[0] + '\''
                        scores += str(word[1])
                    else: 
                        words += ', \'' + word[0] + '\''
                        scores += ', ' + str(word[1])
                print('words:', words)
                print('scores:', scores)
                print()
            print()

In [ ]:
train = Analyzer("stsa.binary.phrases.train")
train.processData()
train.sortData()
train.checkCounting()

In [ ]:
train.lengthAnalysis()

In [ ]:
train.lengthCorrelation()

In [ ]:
train.printFrequent(5)

In [ ]:
#########
## DEV ##
#########
dev = Analyzer("stsa.binary.dev")
dev.processData()
dev.sortData()
dev.checkCounting()

In [ ]:
dev.lengthAnalysis()

In [ ]:
dev.lengthCorrelation()

In [ ]:
dev.printFrequent(5)

In [ ]:
##########
## TEST ##
##########
test = Analyzer("stsa.binary.test")
test.processData()
test.sortData()
test.checkCounting()

In [ ]:
test.lengthCorrelation()

In [ ]:
test.lengthAnalysis()

In [ ]:
test.printFrequent(5)

In [101]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

class ExtendedSentAnal(Analyzer):
    cv = CountVectorizer()
    vectorizer = TfidfVectorizer()
    tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True) 

    def __init__(self, sourcePath):
        Analyzer.__init__(self, sourcePath)
    
    def tfIdfWithCV(self):
        word_count_vector = cv.fit_transform(self.texts)
        print(word_count_vector)
        # sort ascending 
        # df_idf = pd.DataFrame(word_count_vector.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
    
    def tfIdfWithTranformer(self):
        tfidf_transformer.fit(word_count_vector)
        # print(word_count_vector)
        df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
        # sort ascending 
        print(tfidf_transformer.idf_)
        # print(df_idf.sort_values(by=['idf_weights']))


train = ExtendedSentAnal("stsa.binary.phrases.train")
#instantiate CountVectorizer() 
train.tfIdfWithTranformer()
 
# this steps generates word counts for the words in your docs 



# print idf values 

 


[ 9.76616042  7.54153687  8.61348091 ... 10.86477271 10.4593076
 10.4593076 ]


In [75]:
# count matrix 
count_vector=cv.transform(train.texts) 
 
# tf-idf scores 
tf_idf_vector = tfidf_transformer.transform(count_vector)

In [82]:
feature_names = cv.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[0]
 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
dredge,0.433279
nerds,0.400452
revenge,0.366934
clichés,0.332826
filmmakers,0.313861
...,...
fidgeted,0.000000
field,0.000000
fields,0.000000
fiend,0.000000
